# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [ ]:
from utils import get_dataset
import glob
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import tensorflow as tf

%matplotlib inline


In [ ]:
dataset = get_dataset("/home/workspace/home/data/train/*.tfrecord")


## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [ ]:
def display_images(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes. Referenced from 'Explore augmentations.ipynb'
    provided by Udacity.
    """
    # color mapping of classes
    colormap = {
        1: {
            'class': 'vehicle',
            'bbcolor': 'red'
        },
        2: {
            'class': 'pedestrian',
            'bbcolor': 'blue'
        },
        4: {
            'class': 'cyclist',
            'bbcolor': 'green'
        }
    }

    for instance in batch:
        img = instance['image'].numpy()
        bboxes = instance['groundtruth_boxes'].numpy()
        classes = instance['groundtruth_classes'].numpy()

        w, h, _ = img.shape
        bboxes[:, [0, 2]] *= w
        bboxes[:, [1, 3]] *= h

        _, ax = plt.subplots(1, figsize=(10, 10))
        ax.imshow(img)

        for bb, cl in zip(bboxes, classes):
            y1, x1, y2, x2 = bb
            rec = Rectangle((x1, y1), x2 - x1, y2 - y1,
                            facecolor='none', edgecolor=colormap[cl]['bbcolor'])
            ax.add_patch(rec)
        ax.axis('off')
    plt.show()


## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [ ]:
display_images(dataset.shuffle(100).take(10))


## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...

In [ ]:
def class_distribution(batch):
    """
    This function takes a batch from the dataset and displays a histogram
    of the class distribution.
    """
    count_vehicles, count_pedestrians, count_cyclists = 0, 0, 0

    for item in batch:
        classes = item["groundtruth_classes"].numpy()
        for cl in classes:
            if cl == 1:
                count_vehicles += 1
            elif cl == 2:
                count_pedestrians += 1
            elif cl == 4:
                count_cyclists += 1

    fig, ax = plt.subplots()

    classes = ['vehicles', 'pedestrians', 'cyclists']
    counts = [count_vehicles, count_pedestrians, count_cyclists]
    bar_colors = ['red', 'blue', 'green']

    ax.bar(classes, counts, label=classes, color=bar_colors)
    ax.set_ylabel('count of class')
    ax.set_title('Class Distribution')

    plt.show()
    print(f'Count of vehicles = {count_vehicles}')
    print(f'Count of pedestrians = {count_pedestrians}')
    print(f'Count of cyclists = {count_cyclists}')


In [ ]:
class_distribution(dataset.take(1000))
